In [1]:
from database_handling.DataDownload import DataDownloader
from database_handling.DataHandleAndOtherHelpers import DataHandler
from database_handling.DataUpload import DataUploader
from database_handling.DataDelete import DataDeleter
from database_handling.KeycloakLogin import KeycloakLogin
from scrapers.BayerischerRundfunkScraper import BayerischerRundfunkScraper
from text_analysis.NEExtractor import NEExtractor
from text_analysis.Summarizer import Summarizer
from text_analysis.TopicExtractor import TopicExtractor
from text_analysis.Vectorizers import Vectorizer
from selenium.webdriver.common.by import By

import transformers
import json
import gc
import torch

c:\Users\ra76lax\AppData\Local\anaconda3\envs\python_m3_scrapers\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\ra76lax\AppData\Local\anaconda3\envs\python_m3_scrapers\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\ra76lax\AppData\Local\anaconda3\envs\python_m3_scrapers\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
INFO:datasets:PyTorch version 2.2.2 available.


In [2]:
import selenium

In [3]:
scraper = BayerischerRundfunkScraper(headless=False)
scraper.start_browser()

In [4]:
# in the other scrapers the navigate_to function is called in the login() function
# since bayerischer rundfunk does not have a paywall and login, we just navigate to the start page
scraper.navigate_to("https://www.br.de/nachrichten/")

INFO:scrapers.BaseScraper:Navigated to https://www.br.de/nachrichten/


In [5]:
scraper.click_cookie_button()

In [6]:
# at the time of finalizing the scraper for bayerischer rundfunk the website introduced dark mode (only a few days ago). users can choose between light, system, and dark
# I'm very sure the popup for choosing will not show up like this indefineteley, but at some point they will default to system
# therefore the follwing line is used right here rather then being implemented in the actual scraper class somewhere
scraper.driver.find_element(By.CSS_SELECTOR, ".RedesignModal_saveButton__nIBZN").click()

In [9]:
subpage_urls = scraper._get_subpage_urls_on_current_page()


In [10]:
subpage_urls

[]

In [ ]:
all_found_urls = scraper.get_article_urls()

In [ ]:
this_runs_articles = scraper.scrape()

In [2]:
# Open the file and load its content
with open("bayerischer_rundfunk_articles.json", 'r') as file:
    this_runs_articles = json.load(file)

In [3]:
this_runs_articles = this_runs_articles[0]

In [8]:
# Save the responses to a json file
#with open('bayerischer_rundfunk_articles.json', 'w') as f:
#    json.dump(this_runs_articles, f)


In [4]:
# get the token for the database
keycloak_login = KeycloakLogin()
token = keycloak_login.return_token()

In [3]:
from database_handling.DataDownload import DataDownloader

test_url_checker = DataDownloader(token)
status_code = test_url_checker.get_content_rehydrate_status_code_only(url="https://www.spiegel.de/deinspiegel/wie-der-wolf-zum-hund-wurde-der-beginn-einer-wunderbaren-freundschaft-a-4388fdf5-aeef-40f0-9d9c-753f3de38f9c")

In [4]:
status_code

200

In [6]:
import requests

In [8]:
# Construct the full URL for the API request
api_url = f"https://api.m3.ifkw.lmu.de/api/v1/content/rehydrate/"

# Send the HEAD request with the optional query parameters
response = requests.head(api_url, url="https://www.spiegel.de/deinspiegel/wie-der-wolf-zum-hund-wurde-der-beginn-einer-wunderbaren-freundschaft-a-4388fdf5-aeef-40f0-9d9c-753f3de38f9c")

TypeError: head() got multiple values for argument 'url'

In [5]:
# Initialize the data downloader with the provided token
data_downloader = DataDownloader(token)

# Initialize variables for pagination
all_articles_in_db = []
page_size = 100  # Assuming the server returns 100 items per page by default
offset = 0
url = "https://www.br.de/nachrichten/"

# Loop until all pages are retrieved
while True:
    # Fetch articles from the current page
    articles_in_db = data_downloader.get_content(url=url, limit=page_size, offset=offset)
    
    # Check if articles were returned
    if not articles_in_db or 'items' not in articles_in_db:
        break
    
    # Add the current batch of articles to the complete list
    all_articles_in_db.extend(articles_in_db['items'])
    
    # Update the offset for the next page
    offset += page_size
    
    # Check if we have fetched all articles
    if len(all_articles_in_db) >= articles_in_db['count']:
        break

# The total number of articles retrieved
article_count = len(all_articles_in_db)

print(f"Total articles fetched: {article_count}")


Total articles fetched: 737


In [6]:
# instantiate the data handler
data_handler = DataHandler()

In [7]:
# for the articles that are already in the database, only update the last_verification_date
articles_for_last_verifcation_date_update = data_handler.find_scraped_articles_already_in_db(this_runs_articles, all_articles_in_db)
# safe the responses to the last verification date update
responses_to_last_verifcation_date_update = data_handler.patch_last_online_verification_date(token, articles_for_last_verifcation_date_update)
# get the articles that are not yet in the database
articles_not_yet_in_db = data_handler.find_scraped_articles_not_already_in_db(this_runs_articles, all_articles_in_db)
articles_not_yet_in_db_list_of_dicts = [article for article in this_runs_articles if article['url'] in articles_not_yet_in_db]

# Define the number of articles to process and upload per iteration
articles_per_iteration = 10

# Calculate the number of iterations
iterations = len(articles_not_yet_in_db_list_of_dicts) // articles_per_iteration + (len(articles_not_yet_in_db_list_of_dicts) % articles_per_iteration > 0)

responses = []

TypeError: string indices must be integers

In [13]:
import torch
import gc
import json

def clear_gpu_memory():
    """Clears GPU memory and forces garbage collection."""
    torch.cuda.empty_cache()
    gc.collect()

def process_articles_in_batches(text_analysis_class, method_name, articles, batch_size):
    """Process articles in batches using the specified text analysis class and method."""
    processor = text_analysis_class()
    method = getattr(processor, method_name)
    
    for i in range(0, len(articles), batch_size):
        batch = articles[i:i + batch_size]
        batch = method(batch)
        
        # Reassign the processed batch back to the main list
        articles[i:i + batch_size] = batch

        del batch
        clear_gpu_memory()  # Clear memory after each batch
    
    del processor  # Delete the processor instance to free up GPU memory
    clear_gpu_memory()

# Define your batch size
batch_size = len(articles_not_yet_in_db_list_of_dicts)  # Adjust this based on your GPU capacity

# Process with Entity Extractor
print("Starting entity extraction...")
process_articles_in_batches(NEExtractor, 'extract_entities', articles_not_yet_in_db_list_of_dicts, batch_size)
print("Entity extraction completed.")

# Process with Topic Extractor
print("Starting topic extraction...")
process_articles_in_batches(TopicExtractor, 'extract_topics', articles_not_yet_in_db_list_of_dicts, batch_size)
print("Topic extraction completed.")

# Process with Vectorizer
print("Starting vectorization...")
process_articles_in_batches(Vectorizer, 'vectorize', articles_not_yet_in_db_list_of_dicts, batch_size)
print("Vectorization completed.")

# Remove main_text and lead_text from articles to save space before uploading
for article in articles_not_yet_in_db_list_of_dicts:
    article.pop('main_text', None)
    article.pop('lead_text', None)

# Upload each article to the database
print("Beginning article upload...")
responses = []
keycloak_login = KeycloakLogin()
token = keycloak_login.return_token()
data_uploader = DataUploader(token)

for article in articles_not_yet_in_db_list_of_dicts:
    response = data_uploader.post_content(article)
    responses.append(response)

# Save the responses to a JSON file
with open('responses.json', 'w') as f:
    json.dump(responses, f)

print("Article upload completed.")


Starting entity extraction...
2024-08-29 15:47:37,134 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>
Tagger instantiated successfully


Map:   0%|          | 0/742 [00:00<?, ? examples/s]

Entity extraction completed.
Starting topic extraction...


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/742 [00:00<?, ? examples/s]

No sentence-transformers model found with name deepset/gbert-base. Creating a new one with MEAN pooling.


Topic extraction completed.
Starting vectorization...


No sentence-transformers model found with name T-Systems-onsite/german-roberta-sentence-transformer-v2. Creating a new one with MEAN pooling.
No sentence-transformers model found with name xlm-roberta-large. Creating a new one with MEAN pooling.
No sentence-transformers model found with name google/bigbird-roberta-large. Creating a new one with MEAN pooling.
No sentence-transformers model found with name severinsimmler/xlm-roberta-longformer-large-16384. Creating a new one with MEAN pooling.


Map:   0%|          | 0/742 [00:00<?, ? examples/s]

Vectorization completed.
Beginning article upload...
Article upload completed.


In [ ]:
#for i in range(iterations):
#    print("Processing articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(articles_not_yet_in_db_list_of_dicts))
#    # Get the articles for this iteration
#    articles = spiegel_articles_not_yet_in_db_list_of_dicts[i*articles_per_iteration:(i+1)*articles_per_iteration]#
#
#    print("Running text processing on articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(articles_not_yet_in_db_list_of_dicts))
    # Add the summaries, named entities, topics, and vectors to the articles dict
    #articles = summarizer.summarize(articles)
#    articles = entity_extractor.extract_entities(articles)
#    articles = topic_extractor.extract_topics(articles)
#    articles = vectorizer.vectorize(articles)

    # Remove main_text and lead_text from articles
#    for article in articles:
#        article.pop('main_text', None)
#        article.pop('lead_text', None)
        
#    print("Uploading articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(articles_not_yet_in_db_list_of_dicts))
    # Ensure that the token is still valid every n iterations
    # TODO: Tell Mario chuncking was done because I get a new token every 30 uploads to make sure the token is always valid
    # if we do that every 1 upload that takes much longer since it takes ~20 seconds to get a net token/ensure the token is valid#
#    keycloak_login = KeycloakLogin()
#    token = keycloak_login.return_token()
    
    # Loop over articles and put every article into the database
#    data_uploader = DataUploader(token)
    
#    for article in articles:
#        response = data_uploader.post_content(article)
#        responses.append(response)
    
#    print("Processed and uploaded articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(articles_not_yet_in_db_list_of_dicts))
    
        
    # save the responses to a json file
#    with open('responses.json', 'w') as f:
#        json.dump(responses, f)